# Other Statements

In this notebook we'll present several SQL statements and commands so that we can add new skills to our SQl arsenal.

Just like in previous notebooks we create the usual functions and objects

In [2]:
# Usual libraries
import pandas as pd
import psycopg2 as pg2

# Creates the connection to the database
connection = pg2.connect(database = 'premier_league', user = 'postgres', password = 'password')

# Defines get_data function
def get_data(query, rows = 10):

    with connection.cursor() as cursor:
        cursor.execute(query)

        if rows == 'all':
            raw_data = cursor.fetchall()
        else:
            raw_data = cursor.fetchmany(rows) 

        col_names = [col_desc[0] for col_desc in cursor.description]
        data = pd.DataFrame(raw_data, columns = col_names)

    return data

# Creates cursor
cursor = connection.cursor()

Recall that, in the previous notebook, to show the usage of the DELELTE and DROP statements we removed some some rows and columns form the premiere_league database. To have more data to play with, we'll restore the database to its original state.

In [3]:
# Deletes tables
delete_tables = 'DROP TABLE IF EXISTS clubs, players'
cursor.execute(delete_tables)

create_clubs_table = '''
                     CREATE TABLE clubs(
                         club_id SERIAL PRIMARY KEY,
                         name VARCHAR(100) UNIQUE NOT NULL,
                         stadium_name VARCHAR(100), 
                         location VARCHAR(100),
                         times_champion INTEGER CHECK(times_champion >= 0)
                     )
                     '''

create_players_table = '''
                       CREATE TABLE players(
                           player_id SERIAL PRIMARY KEY,
                           first_name VARCHAR(100) NOT NULL,
                           last_name VARCHAR(100) NOT NULL, 
                           club_id INTEGER REFERENCES clubs(club_id),
                           nationality VARCHAR(100) NOT NULL
                       )
                       '''

# Creates tables
cursor.execute(create_clubs_table)
cursor.execute(create_players_table)

Interts the rows into both tables 

In [4]:
# Inserts clubs
clubs = [
        ('Liverpool FC', 'Anfield', 'Liverpool', 19),
        ('Manchester United FC', 'Old Trafford', 'Manchester', 20),
        ('Manchester City FC', 'Etihad Stadium', 'Manchester', 6),
        ('Chelsea FC', 'Stamford Bridge', 'London', 6),
        ('Tottenham Hotspur FC', 'Tottenham Hotspur Stadium', 'London', 2)
        ]

insert_generic_club = '''
                      INSERT INTO clubs(name, stadium_name, location, times_champion)
                      VALUES
                      (%s, %s, %s, %s)
                      '''

for club in clubs:
    cursor.execute(insert_generic_club, club)

# Insterts players
players = [
          ('Mohamed', 'Salah', 1, 'Egyptian'),
          ('Sadio', 'Mane', 1, 'Senegalese'),
          ('Marcus', 'Rashford', 2, 'British'),
          ('Sergio', 'Aguero', 3, 'Argentine'),
          ('Timo', 'Werner', 4, 'German') ,
          ('Harry', 'Kane', 5, 'British')
          ]

insert_player = '''
                INSERT INTO players(first_name, last_name, club_id, Nationality)
                VALUES
                (%s, %s, %s, %s)
                '''
                
for player in players:
    cursor.execute(insert_player, player)

Let's display the tables

In [5]:
clubs = get_data('SELECT * FROM clubs')
players = get_data('SELECT * FROM players')

clubs

,club_id,name,stadium_name,location,times_champion
0,1,Liverpool FC,Anfield,Liverpool,19
1,2,Manchester United FC,Old Trafford,Manchester,20
2,3,Manchester City FC,Etihad Stadium,Manchester,6
3,4,Chelsea FC,Stamford Bridge,London,6
4,5,Tottenham Hotspur FC,Tottenham Hotspur Stadium,London,2


In [6]:
players

,player_id,first_name,last_name,club_id,nationality
0,1,Mohamed,Salah,1,Egyptian
1,2,Sadio,Mane,1,Senegalese
2,3,Marcus,Rashford,2,British
3,4,Sergio,Aguero,3,Argentine
4,5,Timo,Werner,4,German
5,6,Harry,Kane,5,British


## CASE Statement

The CASE statement is very similiar to the _ if, if else, else _ block in other programming languages. It will help us to only excecute a particular set of SQL statements when certain conditions are satisfied. 

## Ex. 1

For example, we can ask whether or not a player plays for the liverpool

In [7]:
query_1 = '''
          SELECT first_name, last_name, 
          CASE 
              WHEN club_id = 1 THEN 'yes'
              ELSE 'no'
          END AS plays_for_liverpool 
          FROM players        
          '''

plays_for_liverpool = get_data(query_1)
plays_for_liverpool

,first_name,last_name,plays_for_liverpool
0,Mohamed,Salah,yes
1,Sadio,Mane,yes
2,Marcus,Rashford,no
3,Sergio,Aguero,no
4,Timo,Werner,no
5,Harry,Kane,no


## COALESCE Function

The COALESCE function accepts an unlimited number or arguments and returns the first not NULL argument. The main purpose of this functions is tu help us deal with the NULL values in a table.

Notice that there are no NULL values in both tables. We'll modify a couple of rows un the clubs table so that there are NULL values in the table.

In [8]:
insert_null_values = '''
                     UPDATE clubs
                     SET times_champion = NULL
                     WHERE name IN ('Chelsea FC', 'Manchester City FC')
                     '''

cursor.execute(insert_null_values)
# Displays the clubs table
clubs = get_data('SELECT * FROM clubs')
clubs

,club_id,name,stadium_name,location,times_champion
0,1,Liverpool FC,Anfield,Liverpool,19.0
1,2,Manchester United FC,Old Trafford,Manchester,20.0
2,5,Tottenham Hotspur FC,Tottenham Hotspur Stadium,London,2.0
3,3,Manchester City FC,Etihad Stadium,Manchester,NaN
4,4,Chelsea FC,Stamford Bridge,London,NaN


## Ex. 3

Imagine that, for some reason, we need to add one to all the times_champion values. Also, let´s suppose that the reason why some clubs have a NULL value in the time_champion field, is because they have never won the premier league (despite we know that Chelsea and Man. City have, indeed, won the league several times). 

Let's see what happens if we just try to make the sum without taking attention to the NULL values in teh table

In [ ]:
query_2 = '''
          SELECT name, 
          CASE 
              WHEN times_champion > 0 THEN 'yes'
              ELSE 'no'
          END AS has_been_champion 
          FROM clubs      
          '''